In [20]:
### This Works!!!

from prefect.blocks.system import Secret

# Save the secret
secret_block = Secret(value='BpNmYUxjlVgb88qTtgtQyWask')
await secret_block.save(name='socrata-app-token', overwrite=True)


UUID('3236e3ae-4b1d-4d59-82fb-30d032ddc4f1')

In [23]:
from prefect.blocks.system import Secret

# Load the secret
secret_block = await Secret.load("socrata-app-token")
loaded_secret = secret_block.get()
print(loaded_secret)

BpNmYUxjlVgb88qTtgtQyWask


In [16]:
import prefect

prefect.__version__

'2.20.1'

In [17]:
from prefect.blocks.system import Secret

In [15]:
from prefect.variables import Variable


# getting from a synchronous context
answer = await Variable.get('life_everything')
print(answer.value)

ValidationError: 1 validation error for Variable
value
  str type expected (type=type_error.str)

In [13]:
from prefect.blocks.system import JSON

json_block = JSON(value={"the_answer": 42})
json_block.save(name="life-the-universe-everything")

<coroutine object Block.save at 0x150048dd0>

In [10]:
from prefect.blocks.system import Secret

# Create a new secret
secret = Secret(name="MY_SECRET", value="super_secret_value")
secret.save()

<coroutine object Block.save at 0x150048190>

In [7]:
import pandas as pd
data = [
        {'name': 'Andy', 'age': '33',},
        {'name': 'Bob', 'age': '10',}
        ]
data = pd.DataFrame(data)

In [9]:
# Path to your CSV file
csv_file = 'the_data_2.csv'

# Append new data to the CSV file
csv_data = data.to_csv(mode='a', index=False, header=False)

csv_data

'Andy,33\nBob,10\n'

In [5]:
from prefect_gcp import GcpCredentials
gcp_credentials_block = GcpCredentials.load("my-cloud-run-pool-push-pool-credentials")

# Extract the JSON string from the SecretDict
credentials_info = gcp_credentials_block.service_account_info.get_secret_value()
credentials_info

AttributeError: 'coroutine' object has no attribute 'service_account_info'

In [1]:
import os
from google.oauth2 import service_account
key_path = os.getenv('GCP_ACCESS_KEY') 
credentials = service_account.Credentials.from_service_account_file(key_path)
credentials

In [6]:
from prefect.variables import Variable

# Set the variable in Prefect Cloud
Variable.set(name="my_variable", value="some_value")

<coroutine object Variable.set at 0x13b4df0d0>

In [5]:
from prefect.client import get_client
get_client()

In [4]:
from prefect.variables import Variable

# setting the variable
variable = Variable.set(name="the_answer", value="42")
variable

/var/folders/0d/9xk1njx10y7ftjfbgf9nsjc00000gn/T/ipykernel_27522/666501201.py:4: RuntimeWarning: coroutine 'Variable.set' was never awaited
  variable = Variable.set(name="the_answer", value="42")


<coroutine object Variable.set at 0x13b4de6c0>

In [2]:
import prefect.settings
prefect.settings.PREFECT_HOME.value()

PosixPath('/Users/annieyuchuan/.prefect')

In [4]:
import asyncio
from prefect_gcp import GcpCredentials

async def load_credentials():
    gcp_credentials_block = await GcpCredentials.load("my-cloud-run-pool-push-pool-credentials")
    return gcp_credentials_block.project_id

project_id = asyncio.run(load_credentials())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [3]:
from prefect_gcp import GcpCredentials
gcp_credentials_block = GcpCredentials.load("my-cloud-run-pool-push-pool-credentials")
gcp_credentials_block.project_id

/var/folders/0d/9xk1njx10y7ftjfbgf9nsjc00000gn/T/ipykernel_90519/2938478317.py:2: RuntimeWarning: coroutine 'Block.load' was never awaited
  gcp_credentials_block = GcpCredentials.load("my-cloud-run-pool-push-pool-credentials")


AttributeError: 'coroutine' object has no attribute 'project_id'

In [1]:
import pandas as pd
from datetime import date, timedelta, datetime

from sodapy import Socrata

from prefect import task, Flow
from prefect.client.schemas.schedules import IntervalSchedule, CronSchedule

from google.cloud import bigquery
from google.oauth2 import service_account

import os
from dotenv import load_dotenv

In [11]:
def fetch_data(wrong_data=123):
    data = [
        {'name': 'Andy', 'age': '33', 'time': datetime.now(), 'random': wrong_data},
        {'name': 'Bob', 'age': '10', 'time': datetime.now(), 'wrong': wrong_data},
        ]
    
    return data
def transform_data(data):
    needed_keys = ['name', 'age', 'time']

    filtered_data = []
    for d in data:
        filtered_dict = {} 
        for k, v in d.items():
            if k in needed_keys: 
                if v is not None and not isinstance(v, str):
                    v = str(v)
                filtered_dict[k] = v 
        filtered_data.append(filtered_dict) 
    return filtered_data

def save_data(data):
    data = pd.DataFrame(data)
    
    # Path to your CSV file
    csv_file = 'the_data.csv'

    # Append new data to the CSV file
    data.to_csv(csv_file, mode='a', index=False, header=False)

In [12]:
data = fetch_data()
filtered_data = transform_data(data)
new_data = pd.DataFrame(filtered_data)
save_data(new_data)

In [33]:
# Load environment variables from .env file
load_dotenv()

os.getenv('SOCRATA_APP_TOKEN')

'BpNmYUxjlVgb88qTtgtQyWask'

In [20]:
# Load environment variables from .env file
load_dotenv()

# set up Socrata client
socrata_client = Socrata(
    'data.sfgov.org',
    app_token=os.getenv('SOCRATA_APP_TOKEN'),
    username=os.getenv('SOCRATA_USERNAME'),
    password=os.getenv('SOCRATA_PASSWORD')
    )

# set up BigQuery client
# Path to your service account key file
key_path = "/Users/annieyuchuan/Desktop/mlops-exploration/mlops-dbt/my_first_dbt_project_local/my-dbt-sa-local-key.json"

# Create credentials using the service account key file
credentials = service_account.Credentials.from_service_account_file(key_path)

# Initialize the BigQuery client with the credentials
bigquery_client = bigquery.Client(
    credentials=credentials, 
    project=credentials.project_id
    )

In [21]:
# Example operation: listing tables in a dataset
dataset_id = "test_dataset"
tables = bigquery_client.list_tables(dataset_id)

for table in tables:
    print(table.table_id)

311_data
gaming
names


In [22]:
def fetch_data(client, data_limit=100):
    # fetch 311 data
    data = client.get(
        dataset_identifier='vw6y-z8j6', 
        limit=data_limit)
    
    return data

In [23]:
def transform_data(data):
    needed_keys = [
        'service_request_id', 'requested_datetime', 'closed_date', 'updated_datetime', 'status_description', 'status_notes',
        'agency_responsible', 'service_name', 'service_subtype', 'service_details', 'address', 'street', 'supervisor_district',
        'neighborhoods_sffind_boundaries', 'police_district', 'lat', 'long', 'source'
    ]

    # 1. get the interpretable data by keys
    # 2. convert the datatype of values to string
    filtered_data = []
    for d in data:
        filtered_dict = {} 
        for k, v in d.items():
            if k in needed_keys: 
                if v is not None and not isinstance(v, str):
                    v = str(v)
                filtered_dict[k] = v 
        filtered_data.append(filtered_dict) 

    return filtered_data

In [24]:
def insert_record_one_by_one(client, table_id, records):
    failed_records = []
    
    for record in records:
        # Insert a single record into BigQuery
        errors = client.insert_rows_json(table_id, [record])
        
        if errors:
            # Append the record and errors to the failed_records list
            failed_records.append({'record': record, 'errors': errors})
        else:
            print(f"Record inserted successfully: {record}")

    return failed_records

def load_data_to_bigquery(client, data):
    table_id = 'my-tenth-project-432516.test_dataset.311_data'
    
    # Insert records and get failed records
    failed_records = insert_record_one_by_one(client, table_id, data)
    
    # Generate a report of failed records
    if failed_records:
        with open('failed_records_report.txt', 'w') as f:
            for entry in failed_records:
                f.write(f"Record: {entry['record']}\n")
                f.write(f"Errors: {entry['errors']}\n")
                f.write("\n---\n\n")
        print("Report of failed records has been written to 'failed_records_report.txt'.")
    else:
        print("All records were inserted successfully.")


In [25]:
data_limit=10
data = fetch_data(socrata_client, data_limit)
transformed_data = transform_data(data)
transformed_data[0]

{'service_request_id': '1000040',
 'requested_datetime': '2011-11-08T10:10:55.000',
 'closed_date': '2011-11-11T06:07:07.000',
 'updated_datetime': '2011-11-11T06:07:07.000',
 'status_description': 'Closed',
 'status_notes': 'See Notes tab for more details',
 'agency_responsible': 'DPW Ops Queue',
 'service_name': 'Street and Sidewalk Cleaning',
 'service_subtype': 'General Cleaning',
 'service_details': 'Other Loose Garbage',
 'address': 'Not associated with a specific address',
 'lat': '0.0',
 'long': '0.0',
 'source': 'Web'}

In [26]:
load_data_to_bigquery(bigquery_client, transformed_data)

Record inserted successfully: {'service_request_id': '1000040', 'requested_datetime': '2011-11-08T10:10:55.000', 'closed_date': '2011-11-11T06:07:07.000', 'updated_datetime': '2011-11-11T06:07:07.000', 'status_description': 'Closed', 'status_notes': 'See Notes tab for more details', 'agency_responsible': 'DPW Ops Queue', 'service_name': 'Street and Sidewalk Cleaning', 'service_subtype': 'General Cleaning', 'service_details': 'Other Loose Garbage', 'address': 'Not associated with a specific address', 'lat': '0.0', 'long': '0.0', 'source': 'Web'}
Record inserted successfully: {'service_request_id': '1000203', 'requested_datetime': '2011-11-08T12:12:49.000', 'closed_date': '2011-11-08T13:07:08.000', 'updated_datetime': '2011-11-08T13:07:08.000', 'status_description': 'Closed', 'status_notes': 'See Notes tab for more details', 'agency_responsible': 'DPW Ops Queue', 'service_name': 'Street and Sidewalk Cleaning', 'service_subtype': 'General Cleaning', 'service_details': 'Other Loose Garbage

# Create sample data

In [ ]:
# pd.read_csv("/Users/annieyuchuan/Downloads/311_Cases_20240815.csv").head().to_csv("data/311_samples.csv", index=False)

In [ ]:
# cols = [
#         'service_request_id', 'requested_datetime', 'closed_date', 'updated_datetime', 'status_description', 'status_notes',
#         'agency_responsible', 'service_name', 'service_subtype', 'service_details', 'address', 'street', 'supervisor_district',
#         'neighborhoods_sffind_boundaries', 'police_district', 'lat', 'long', 'point', 'point_geom', 'source'
#         ]

# df = pd.read_csv("data/311_samples.csv")
# df

,CaseID,Opened,Closed,Updated,Status,Status Notes,Responsible Agency,Category,Request Type,Request Details,...,DELETE - HSOC Zones,Fix It Zones as of 2018-02-07,"CBD, BID and GBD Boundaries as of 2017",Central Market/Tenderloin Boundary,"Areas of Vulnerability, 2016",Central Market/Tenderloin Boundary Polygon - Updated,HSOC Zones as of 2018-06-05,OWED Public Spaces,Parks Alliance CPSI (27+TL sites),Neighborhoods
0,16987193,07/01/2023 03:44:00 PM,07/04/2023 10:29:00 AM,07/04/2023 10:29:00 AM,Closed,Case Resolved,DPW Ops Queue,Street and Sidewalk Cleaning,Bulky Items,Boxed or Bagged Items,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,105.0
1,16987195,07/01/2023 03:44:00 PM,07/02/2023 04:30:33 PM,07/02/2023 04:30:33 PM,Closed,Case Resolved - Pickup completed,Recology_Overflowing,Street and Sidewalk Cleaning,City_garbage_can_overflowing,City_garbage_can_overflowing,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,84.0
2,16987192,07/01/2023 03:44:08 PM,07/02/2023 04:24:36 PM,07/02/2023 04:24:36 PM,Closed,Case Resolved - Pickup completed,Recology_Overflowing,Street and Sidewalk Cleaning,City_garbage_can_overflowing,City_garbage_can_overflowing,...,NaN,25.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,25.0
3,16987194,07/01/2023 03:44:16 PM,07/02/2023 04:28:34 PM,07/02/2023 04:28:34 PM,Closed,Case Resolved - Pickup completed,Recology_Overflowing,Street and Sidewalk Cleaning,City_garbage_can_overflowing,City_garbage_can_overflowing,...,NaN,NaN,NaN,NaN,1.0,NaN,5.0,NaN,NaN,37.0
4,16987198,07/01/2023 03:45:00 PM,07/01/2023 05:17:21 PM,07/01/2023 05:17:21 PM,Closed,Case Resolved - Officer responded to request u...,Parking Enforcement Dispatch Queue,Parking Enforcement,Blocking_Driveway_Cite_Only,white - white -,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,86.0


In [ ]:
# df[['Source', 'Media URL']]

,Source,Media URL
0,Mobile/Open311,NaN
1,Mobile/Open311,NaN
2,Mobile/Open311,NaN
3,Mobile/Open311,NaN
4,Phone,NaN


In [ ]:
# df.columns

Index(['CaseID', 'Opened', 'Closed', 'Updated', 'Status', 'Status Notes',
       'Responsible Agency', 'Category', 'Request Type', 'Request Details',
       'Address', 'Street', 'Supervisor District', 'Neighborhood',
       'Police District', 'Latitude', 'Longitude', 'Point', 'point_geom',
       'Source', 'Media URL', 'BOS_2012', 'data_as_of', 'data_loaded_at',
       'SF Find Neighborhoods', 'Current Police Districts',
       'Current Supervisor Districts', 'Analysis Neighborhoods',
       'DELETE - Supervisor Districts', 'DELETE - Fire Prevention Districts',
       'DELETE - Current Police Districts', 'DELETE - Zip Codes',
       'DELETE - Police Districts', 'DELETE - Neighborhoods',
       'DELETE - Neighborhoods_from_fyvs_ahh9', 'DELETE - 2017 Fix It Zones',
       'DELETE - SF Find Neighborhoods',
       'Civic Center Harm Reduction Project Boundary',
       'DELETE - Current Supervisor Districts',
       'Fix It Zones as of 2017-11-06 ', 'Invest In Neighborhoods (IIN) Areas',
  

In [ ]:
# old_cols = [
#     'CaseID', 'Opened', 'Closed', 'Updated', 'Status', 'Status Notes', 'Responsible Agency', 'Category', 
#     'Request Type', 'Request Details', 'Address', 'Street', 'Supervisor District', 'Neighborhood', 
#     'Police District', 'Latitude', 'Longitude', 'Point', 'point_geom', 'Source',
#     ]

In [ ]:
# old_cols, len(old_cols)

(['CaseID',
  'Opened',
  'Closed',
  'Updated',
  'Status',
  'Status Notes',
  'Responsible Agency',
  'Category',
  'Request Type',
  'Request Details',
  'Address',
  'Street',
  'Supervisor District',
  'Neighborhood',
  'Police District',
  'Latitude',
  'Longitude',
  'Point',
  'point_geom',
  'Source'],
 20)

In [ ]:
# cols, len(cols)

(['service_request_id',
  'requested_datetime',
  'closed_date',
  'updated_datetime',
  'status_description',
  'status_notes',
  'agency_responsible',
  'service_name',
  'service_subtype',
  'service_details',
  'address',
  'street',
  'supervisor_district',
  'neighborhoods_sffind_boundaries',
  'police_district',
  'lat',
  'long',
  'point',
  'point_geom',
  'source'],
 20)

In [ ]:
# for o, n in zip(old_cols, cols):
#     print(f"old: {o}, new: {n}")

old: CaseID, new: service_request_id
old: Opened, new: requested_datetime
old: Closed, new: closed_date
old: Updated, new: updated_datetime
old: Status, new: status_description
old: Status Notes, new: status_notes
old: Responsible Agency, new: agency_responsible
old: Category, new: service_name
old: Request Type, new: service_subtype
old: Request Details, new: service_details
old: Address, new: address
old: Street, new: street
old: Supervisor District, new: supervisor_district
old: Neighborhood, new: neighborhoods_sffind_boundaries
old: Police District, new: police_district
old: Latitude, new: lat
old: Longitude, new: long
old: Point, new: point
old: point_geom, new: point_geom
old: Source, new: source


In [ ]:
# # Create a dictionary to map old column names to new column names
# rename_dict = dict(zip(old_cols, cols))

# # Rename the columns in the DataFrame
# df.rename(columns=rename_dict, inplace=True)

# # Output the renamed DataFrame
# df

,service_request_id,requested_datetime,closed_date,updated_datetime,status_description,status_notes,agency_responsible,service_name,service_subtype,service_details,...,DELETE - HSOC Zones,Fix It Zones as of 2018-02-07,"CBD, BID and GBD Boundaries as of 2017",Central Market/Tenderloin Boundary,"Areas of Vulnerability, 2016",Central Market/Tenderloin Boundary Polygon - Updated,HSOC Zones as of 2018-06-05,OWED Public Spaces,Parks Alliance CPSI (27+TL sites),Neighborhoods
0,16987193,07/01/2023 03:44:00 PM,07/04/2023 10:29:00 AM,07/04/2023 10:29:00 AM,Closed,Case Resolved,DPW Ops Queue,Street and Sidewalk Cleaning,Bulky Items,Boxed or Bagged Items,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,105.0
1,16987195,07/01/2023 03:44:00 PM,07/02/2023 04:30:33 PM,07/02/2023 04:30:33 PM,Closed,Case Resolved - Pickup completed,Recology_Overflowing,Street and Sidewalk Cleaning,City_garbage_can_overflowing,City_garbage_can_overflowing,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,84.0
2,16987192,07/01/2023 03:44:08 PM,07/02/2023 04:24:36 PM,07/02/2023 04:24:36 PM,Closed,Case Resolved - Pickup completed,Recology_Overflowing,Street and Sidewalk Cleaning,City_garbage_can_overflowing,City_garbage_can_overflowing,...,NaN,25.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,25.0
3,16987194,07/01/2023 03:44:16 PM,07/02/2023 04:28:34 PM,07/02/2023 04:28:34 PM,Closed,Case Resolved - Pickup completed,Recology_Overflowing,Street and Sidewalk Cleaning,City_garbage_can_overflowing,City_garbage_can_overflowing,...,NaN,NaN,NaN,NaN,1.0,NaN,5.0,NaN,NaN,37.0
4,16987198,07/01/2023 03:45:00 PM,07/01/2023 05:17:21 PM,07/01/2023 05:17:21 PM,Closed,Case Resolved - Officer responded to request u...,Parking Enforcement Dispatch Queue,Parking Enforcement,Blocking_Driveway_Cite_Only,white - white -,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,86.0


In [ ]:
# df[cols].to_csv("data/311_samples.csv", index=False)

In [35]:
df = pd.read_csv("data/311_samples.csv").drop(['point','point_geom'], axis=1)
df.to_csv("data/311_samples.csv", index=False)

In [36]:
df

,service_request_id,requested_datetime,closed_date,updated_datetime,status_description,status_notes,agency_responsible,service_name,service_subtype,service_details,address,street,supervisor_district,neighborhoods_sffind_boundaries,police_district,lat,long,source
0,16987193,07/01/2023 03:44:00 PM,07/04/2023 10:29:00 AM,07/04/2023 10:29:00 AM,Closed,Case Resolved,DPW Ops Queue,Street and Sidewalk Cleaning,Bulky Items,Boxed or Bagged Items,"1507 VALLEJO ST, SAN FRANCISCO, CA, 94109",VALLEJO ST,3.0,Polk Gulch,NORTHERN,37.796779,-122.422252,Mobile/Open311
1,16987195,07/01/2023 03:44:00 PM,07/02/2023 04:30:33 PM,07/02/2023 04:30:33 PM,Closed,Case Resolved - Pickup completed,Recology_Overflowing,Street and Sidewalk Cleaning,City_garbage_can_overflowing,City_garbage_can_overflowing,"4125 25TH ST, SAN FRANCISCO, CA, 94114 (Virtual)",25TH ST,8.0,Noe Valley,MISSION,37.749757,-122.432339,Mobile/Open311
2,16987192,07/01/2023 03:44:08 PM,07/02/2023 04:24:36 PM,07/02/2023 04:24:36 PM,Closed,Case Resolved - Pickup completed,Recology_Overflowing,Street and Sidewalk Cleaning,City_garbage_can_overflowing,City_garbage_can_overflowing,"1603 HAIGHT ST, SAN FRANCISCO, CA, 94117",HAIGHT ST,5.0,Haight Ashbury,PARK,37.769687,-122.448837,Mobile/Open311
3,16987194,07/01/2023 03:44:16 PM,07/02/2023 04:28:34 PM,07/02/2023 04:28:34 PM,Closed,Case Resolved - Pickup completed,Recology_Overflowing,Street and Sidewalk Cleaning,City_garbage_can_overflowing,City_garbage_can_overflowing,"1905 15TH ST, SAN FRANCISCO, CA, 94114",15TH ST,8.0,Mission Dolores,MISSION,37.766188,-122.426708,Mobile/Open311
4,16987198,07/01/2023 03:45:00 PM,07/01/2023 05:17:21 PM,07/01/2023 05:17:21 PM,Closed,Case Resolved - Officer responded to request u...,Parking Enforcement Dispatch Queue,Parking Enforcement,Blocking_Driveway_Cite_Only,white - white -,"1119 MENDELL ST, SAN FRANCISCO, CA, 94124",MENDELL ST,10.0,Bayview,BAYVIEW,37.736130,-122.388788,Phone
